In [1]:
# import necessary packages
import pandas as pd
import numpy as np
from sklearn import preprocessing
import statsmodels.api as sm
from sklearn import linear_model

In [8]:
data = pd.read_csv('all_year_with_target.csv')
data.describe()

,year,Gross domestic product (GDP) by state,Per capita real GDP by state,Quantity indexes for real GDP by state,Real GDP by state,pce Clothing and footwear,pce Food and beverages purchased for off-premises consumption,pce Furnishings and durable household equipment,pce Gasoline and other energy goods,pce Motor vehicles and parts,...,State governments,Equals: Disposable personal income,Personal current taxes,contract_establish,end_establish,expand_establish,net_change,open_establish,total_contract,total_expand
count,1017.000000,1.017000e+03,966.000000,1017.000000,1.017000e+03,1017.000000,1017.000000,1017.000000,1017.000000,1017.000000,...,1.017000e+03,1.017000e+03,1.017000e+03,1.017000e+03,1.017000e+03,1.017000e+03,1.017000e+03,1.017000e+03,1.017000e+03,1.017000e+03
mean,2006.525074,2.646932e+05,46974.765010,97.358486,2.761757e+05,981.526057,2432.219272,822.655851,1068.108161,1324.824975,...,5.103276e+06,1.971238e+08,2.646635e+07,9.280420e+05,2.268305e+05,9.536988e+05,3.998977e+04,2.411634e+05,1.154872e+06,1.194862e+06
std,5.758904,3.349438e+05,17064.875619,11.964398,3.357822e+05,198.589474,506.358814,168.080946,497.768275,226.025430,...,8.865942e+06,2.430566e+08,3.653654e+07,1.028917e+06,2.948181e+05,1.074905e+06,1.685001e+05,3.040317e+05,1.316920e+06,1.373351e+06
min,1997.000000,1.476000e+04,28368.000000,61.365000,1.952000e+04,603.000000,1483.000000,395.000000,319.000000,662.000000,...,0.000000e+00,1.025367e+07,1.392771e+06,1.092880e+05,2.017800e+04,1.098140e+05,-1.465756e+06,2.167600e+04,1.320520e+05,1.359560e+05
25%,2002.000000,6.409800e+04,38857.000000,89.874000,6.729400e+04,848.000000,2023.000000,707.000000,707.000000,1181.000000,...,9.587070e+05,4.729874e+07,5.450677e+06,2.546360e+05,5.937600e+04,2.599600e+05,1.152000e+03,6.572400e+04,3.152760e+05,3.233280e+05
50%,2007.000000,1.610450e+05,43732.000000,100.000000,1.696640e+05,941.000000,2417.000000,812.000000,981.000000,1322.000000,...,2.462547e+06,1.214731e+08,1.421030e+07,6.313720e+05,1.328940e+05,6.416600e+05,2.326600e+04,1.434660e+05,7.583500e+05,7.822120e+05
75%,2012.000000,3.203810e+05,50294.500000,104.570000,3.487080e+05,1075.000000,2720.000000,929.000000,1311.000000,1460.000000,...,6.578645e+06,2.423695e+08,3.295209e+07,1.125474e+06,2.581080e+05,1.150120e+06,8.233000e+04,2.728580e+05,1.377686e+06,1.418240e+06
max,2016.000000,2.656080e+06,172523.000000,158.916000,2.333841e+06,1888.000000,4272.000000,1486.000000,4098.000000,2489.000000,...,8.548814e+07,1.894449e+09,3.182417e+08,7.231216e+06,2.167120e+06,7.187654e+06,1.077746e+06,2.450032e+06,9.356028e+06,9.430774e+06


In [3]:
data.head()

,GeoName,year,Compensation of employees,Gross domestic product (GDP) by state,Gross operating surplus,Per capita real GDP by state,Quantity indexes for real GDP by state,Real GDP by state,Subsidies,Taxes on production and imports,...,Equals: Disposable personal income,Personal current taxes,state,contract_establish,end_establish,expand_establish,net_change,open_establish,total_contract,total_expand
0,Alabama,1997,61240,104309.0,36814,31465.0,81.042,137437.0,-328,6582,...,83209782.0,10612417.0,Alabama,648756,138914,654734,41822,174758,787670,829492
1,Alaska,1997,12205,25763.0,11234,62577.0,75.935,38358.0,-62,2386,...,15203610.0,1844364.0,Alaska,158148,37718,168904,15276,42238,195866,211142
2,Arizona,1997,69886,132795.0,54249,34485.0,67.195,163353.0,-383,9044,...,94533284.0,12903928.0,Arizona,833670,228694,1013832,198484,247016,1062364,1260848
3,Arkansas,1997,32852,60248.0,23415,30470.0,79.178,79256.0,-298,4279,...,46699786.0,5709269.0,Arkansas,371418,86722,412804,49658,94994,458140,507798
4,California,1997,574394,1082051.0,441963,41314.0,70.134,1342133.0,-5725,71419,...,761138752.0,120765010.0,California,6281922,2167120,7002930,605040,2051152,8449042,9054082


## Running General Regression

In [9]:
# using expand_establish as the target
# 'Other medical care benefits', 'Other unemployment compensation',
data = data.dropna()
Y = data['expand_establish']
X = data.drop(['expand_establish', 'state', 'GeoName', 'year', 'Compensation of employees',
              'Gross operating surplus', 'Subsidies', 'Taxes on production and imports', 
               'Taxes on production and imports less subsidies', 'Value of inventory change', 
               'Unemployment compensation for railroad employees', 'Other assistance to veterans 1',
               'Corn', 'Oats', 'Other grains', 'Soybeans', 'Forest and maple products', 'Fruits and nuts',
               'Total grains', 'Plus: Statistical adjustment', 'Plus: Value of inventory change',
               'Value of inventory change: crops', 'Value of inventory change: livestock', 
               'Value of inventory change: materials and supplies'
              ], axis=1)

In [22]:
# inout issues with compensations of employees, gross operating surplus, subsidies, taxes on production/less subsidies
X = X.iloc[:, 0:10]

In [23]:
# normalizing the dataset
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X)
X_normalized = pd.DataFrame(np_scaled)
X_normalized.head()

,0,1,2,3,4,5,6,7,8,9
0,0.036644,0.021484,0.201710,0.053911,0.165877,0.106305,0.104491,0.063244,0.250684,0.081818
1,0.004502,0.237307,0.149358,0.008613,0.185624,0.331378,0.178735,0.116962,0.149973,0.180682
2,0.048300,0.042433,0.059764,0.065760,0.127962,0.085411,0.229148,0.079915,0.251232,0.104545
3,0.018614,0.014582,0.182602,0.027311,0.142970,0.012830,0.099908,0.082032,0.213465,0.153409
4,0.436739,0.089806,0.089891,0.604693,0.267773,0.073680,0.131072,0.034665,0.110016,0.125000


In [24]:
model = sm.OLS(Y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       expand_establish   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.985
Method:                 Least Squares   F-statistic:                     6351.
Date:                Mon, 12 Nov 2018   Prob (F-statistic):               0.00
Time:                        18:28:27   Log-Likelihood:                -13033.
No. Observations:                 966   AIC:                         2.609e+04
Df Residuals:                     956   BIC:                         2.613e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
===================================================================================================================================
                                                                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------
Gross domestic product (GDP) by state                              -4.2956      0.141    -30.485      0.000      -4.572      -4.019
Per capita real GDP by state                                       -1.8753      0.431     -4.355      0.000      -2.720      -1.030
Quantity indexes for real GDP by state                            907.1826    536.719      1.690      0.091    -146.101    1960.466
Real GDP by state                                                   7.4168      0.140     52.929      0.000       7.142       7.692
pce   Clothing and footwear                                      -356.2813     50.621     -7.038      0.000    -455.623    -256.940
pce   Food and beverages purchased for off-premises consumption    68.5856     18.975      3.615      0.000      31.348     105.823
pce   Furnishings and durable household equipment                  94.5965     55.485      1.705      0.089     -14.290     203.483
pce   Gasoline and other energy goods                             -47.7401     15.500     -3.080      0.002     -78.158     -17.323
pce   Motor vehicles and parts                                    186.5191     27.776      6.715      0.000     132.010     241.029
pce   Other durable goods                                        -105.4317     70.581     -1.494      0.136    -243.943      33.080
==============================================================================
Omnibus:                      448.288   Durbin-Watson:                   1.672
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            16530.051
Skew:                           1.442   Prob(JB):                         0.00
Kurtosis:                      23.059   Cond. No.                     5.68e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.68e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [25]:
#using sklearn instead of stats models
lm = linear_model.LinearRegression()
model = lm.fit(X,Y)

In [26]:
predictions = lm.predict(X)
#print(predictions)[0:5]

In [27]:
lm.score(X,Y)

0.973610736685525

In [28]:
lm.coef_

array([  -4.20234235,   -1.85323604, -193.27476463,    7.33224573,
       -397.25719891,   53.69246977,  118.42229521,  -37.88287215,
        154.93517445,  -38.94544452])